# Building Autonomous Trader using mt5b3

### Installing mt5b3
### 1. Download it from https://github.com/paulo-al-castro/mt5b3/
### 2. unzip it in a proper fold
### 3. Install Metatrader 5 (https://www.metatrader5.com/)
### 4. Install python package Metatrader5 using pip
####   Use: pip install MetaTrader5 ... or Use sys package




In [1]:
# installing Metatrader5 using sys
import sys
# python MetaTrader5
!{sys.executable} -m pip install Metatrader5
#mt5b3 
!{sys.executable} -m pip install mt5b3

  Using cached mt5b3-0.0.2-py3-none-any.whl (26 kB)


In [2]:
import mt5b3 as b3
import pandas as pd
connected=b3.connect()
if connected:
    print('Ok!! It is connected to B3 exchange!!')
else:
    print('Something went wrong! It is NOT connected to B3!!')



Ok!! It is connected to B3 exchange!!


## A Dummy Trader!

In [2]:
import numpy.random as rand

class DummyTrader:
    def __init__(self):
        pass

    def setup(self,dbars):
        print('just getting started!')

    def trade(self,ops,dbars):
        orders=[] 
        assets=ops['assets']
        for asset in assets:
            if rand.randint(2)==1:     
                order=b3.buyOrder(asset,100)
            else:
            	order=b3.sellOrder(asset,100)
            orders.append(order)
        return orders
    
    def ending(self,dbars):
        print('Ending stuff')

trader=DummyTrader()
print(trader)

## Setup and run a backtest!

In [3]:
# sets Backtest options 
prestart=b3.date(2018,12,10)
start=b3.date(2019,1,10)
end=b3.date(2019,2,27)
capital=100000
results_file='data_equity_file.csv'
verbose=False 
assets=['PETR4']
# Use True if you want debug information for your Trader 
#sets the backtest setup
period=b3.DAILY 
 # it may be b3.INTRADAY (one minute interval)
bts=b3.backtest.set(assets,prestart,start,end,period,capital,results_file,verbose)

# Running the backtest
df= b3.backtest.run(trader,bts)   
# run calls the Trader. setup and trade (once for each bar)

#print the results
print(df)

b3.backtest.evaluate(df)

just getting started!
End of backtest with  33  bars,  saving equity file in  data_equity_file.csv
         date   balance    equity  load
0  2019-01-11  102427.0  100000.0   0.0
1  2019-01-14  100014.0  100014.0   0.0
2  2019-01-15  102425.0  100014.0   0.0
3  2019-01-16  100015.0  100015.0   0.0
4  2019-01-17  102458.0  100015.0   0.0
5  2019-01-18   99992.0   99992.0   0.0
6  2019-01-21  102471.0   99992.0   0.0
7  2019-01-22  100031.0  100031.0   0.0
8  2019-01-23   97562.0  100031.0   0.0
9  2019-01-24   95082.0  100042.0   0.0
10 2019-01-28   97487.0   99892.0   0.0
11 2019-01-29   95024.0   99950.0   0.0
12 2019-01-30   92536.0  100000.0   0.0
13 2019-01-31   95020.0   99988.0   0.0
14 2019-02-01   92515.0  100030.0   0.0
15 2019-02-04   89987.0  100099.0   0.0
16 2019-02-05   92517.0  100107.0   0.0
17 2019-02-06   90042.0   99942.0   0.0
18 2019-02-07   87606.0   99786.0   0.0
19 2019-02-08   85166.0   99806.0   0.0
20 2019-02-11   82754.0   99638.0   0.0
21 2019-02-12   80257

## Let's create a Simple Algorithmic Trader 

In [4]:
## Defines the Trader


class MonoAssetTrader(b3.Trader):
    def trade(self,bts,dbars):
        assets=dbars.keys()
        asset=list(assets)[0]
        orders=[]
        bars=dbars[asset]
        curr_shares=b3.backtest.getShares(bts,asset)
        # number of shares that you can buy
        free_shares=b3.backtest.getAfforShares(bts,dbars,asset)
        rsi=b3.tech.rsi(bars)
        if rsi>=70 and free_shares>0: 
            order=b3.buyOrder(asset,free_shares)
        elif  rsi<70 and curr_shares>0:
            order=b3.sellOrder(asset,curr_shares)
        else:
            order=None
        if order!=None:
                orders.append(order)
        return orders    

trader=MonoAssetTrader()
print(trader)


## Setup and check a backtest!

In [5]:
# sets Backtest options 
prestart=b3.date(2018,12,10)
start=b3.date(2019,1,10)
end=b3.date(2019,2,27)
capital=1000000
results_file='data_equity_file.csv'
verbose=False 
assets=['PETR4']
# Use True if you want debug information for your Trader 
#sets the backtest setup
period=b3.DAILY 
 # it may be b3.INTRADAY (one minute interval)
bts=b3.backtest.set(assets,prestart,start,end,period,capital,results_file,verbose)
if b3.backtest.checkBTS(bts): # check if the backtest setup is ok!
    print('Backtest Setup is Ok!')
else:
    print('Backtest Setup is NOT Ok!')



Backtest Setup is Ok!


## Run the Backtest

In [6]:

# Running the backtest
df= b3.backtest.run(trader,bts)   
# run calls the Trader. setup and trade (once for each bar)

#print the results
print(df)


End of backtest with  33  bars,  saving equity file in  data_equity_file.csv
         date    balance     equity  load
0  2019-01-11    12211.0  1000000.0   0.0
1  2019-01-14      146.0   994302.0   0.0
2  2019-01-15      146.0   993478.0   0.0
3  2019-01-16      146.0   993066.0   0.0
4  2019-01-17  1006662.0  1006662.0   0.0
5  2019-01-18  1006662.0  1006662.0   0.0
6  2019-01-21    -4770.0  1006662.0   0.0
7  2019-01-22    -4770.0   990750.0   0.0
8  2019-01-23    -4770.0  1002582.0   0.0
9  2019-01-24    -4770.0  1007070.0   0.0
10 2019-01-28    -4770.0   976470.0   0.0
11 2019-01-29    -4770.0  1000134.0   0.0
12 2019-01-30    -4770.0  1010334.0   0.0
13 2019-01-31  1008702.0  1008702.0   0.0
14 2019-02-01  1008702.0  1008702.0   0.0
15 2019-02-04  1008702.0  1008702.0   0.0
16 2019-02-05  1008702.0  1008702.0   0.0
17 2019-02-06  1008702.0  1008702.0   0.0
18 2019-02-07  1008702.0  1008702.0   0.0
19 2019-02-08  1008702.0  1008702.0   0.0
20 2019-02-11  1008702.0  1008702.0   0.0

## Evaluate the Backtest result

In [7]:
# evaluates the backtest results
b3.backtest.evaluate(df)



 -----------------------   Backtest Report  ------------------------------- 

Total Return (%)=0.87 in 32 bars 
Average Bar Return (%)=0.03  
Std Deviation of returns (%) =0.8515

 ----------------------        End of Report     -------------------------------- 



## Evaluating Backtesting results

The method backtest.run creates a data file with the name given in the backtest setup (bts) 

This will give you a report about the trader performance

We need ot note that it is hard to perform meaningful evaluations using backtest. There are many pitfalls to avoid and it may be easier to get trading robots with great performance in backtest, but that perform really badly in real operations. 

More about that in mt5b3 backtest evaluation chapter.

For a deeper discussion, we suggest:
Is it a great Autonomous Trading Strategy or you are just fooling yourself Bernardini,M. and Castro, P.A.L

In order to analyze the trader's backtest, you may use :

b3.backtest.evaluateFile(fileName)  #fileName is the name of file generated by the backtest
or
b3.bactest.evaluate(df)  # df is the dataframe returned by b3.backtest.run


# Another Example: Multiasset Trader

In [8]:
import mt5b3 as b3

class MultiAssetTrader(b3.Trader):
    def trade(self,bts,dbars):
        assets=dbars.keys()
        orders=[]
        for asset in assets:
            bars=dbars[asset]
            curr_shares=b3.backtest.getShares(bts,asset)
            money=b3.backtest.getBalance(bts)/len(assets) # divide o saldo em dinheiro igualmente entre os ativos
            # number of shares that you can buy of asset 
            free_shares=b3.backtest.getAfforShares(bts,dbars,asset,money)
            rsi=b3.tech.rsi(bars)
            if rsi>=70 and free_shares>0: 
                order=b3.buyOrder(asset,free_shares)
            elif  rsi<70 and curr_shares>0:
                order=b3.sellOrder(asset,curr_shares)
            else:
                order=None
            if order!=None:
                orders.append(order)
        return orders    


trader=MultiAssetTrader()
print(trader)
 

## Setuping for Multiple Assets

In [9]:
import pandas as pd
connected=b3.connect()
if connected:
    print('Ok!! It is connected to B3 exchange!!')
else:
    print('Something went wrong! It is NOT connected to B3!!')

# sets Backtest options 
prestart=b3.date(2020,5,4)
start=b3.date(2020,5,6)
end=b3.date(2020,9,21)
capital=10000000
results_file='data_equity_file.csv'
verbose=False 
assets=['PETR4','BBAS3','BBDC4','ITUB4','VALE3']
# Use True if you want debug information for your Trader 
#sets the backtest setup
period=b3.INTRADAY 

bts=b3.backtest.set(assets,prestart,start,end,period,capital,results_file,verbose)
if b3.backtest.checkBTS(bts): # check if the backtest setup is ok!
    print('Backtest Setup is Ok!')
else:
    print('Backtest Setup is NOT Ok!')

Ok!! It is connected to B3 exchange!!
Backtest Setup is Ok!


## Run and evaluate the backtest

In [10]:


# Running the backtest
df= b3.backtest.run(trader,bts)   
# run calls the Trader. setup and trade (once for each bar)

# evaluates the backtest results
b3.backtest.evaluate(df)

End of backtest with  96  bars,  saving equity file in  data_equity_file.csv

 -----------------------   Backtest Report  ------------------------------- 

Total Return (%)=5.89 in 95 bars 
Average Bar Return (%)=0.06  
Std Deviation of returns (%) =0.7426

 ----------------------        End of Report     -------------------------------- 

